In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lcs import *
import networkx as nx

In [ ]:
G = nx.karate_club_graph()
A = nx.adjacency_matrix(G, weight=None).todense()
n = A.shape[0]

In [ ]:
b0 = 0.5
a = 0.01
alpha = 1
max_iter = 100
tol = 1e-3

cf1 = lambda nu, b: 1 - (1 - b) ** nu
cf2 = lambda nu, b: b * (nu >= 2)

gamma = 0.1
b = 0.03
rho0 = 1
tmax = 1000

realizations = 100

x0 = np.zeros(n)
x0[list(random.sample(range(n), int(rho0 * n)))] = 1
c1 = cf1(np.arange(n), b)

In [ ]:
# mean fitting
mode = "mean"
ipn_c1 = 0
for _ in range(realizations):
    x = contagion_process(A, gamma, c1, x0, tmin=0, tmax=tmax)
    ipn_c1 += infections_per_node(x, mode) / realizations

f = lambda b: ipn_func(b, ipn_c1, cf2, gamma, A, rho0, 100, tmax, mode)
b1, bvec1, fvec1 = robbins_monro_solve(
    f, b0, a, alpha, max_iter, tol, verbose=True, loss="function", return_values=True
)

# median fitting
mode = "median"
ipn_c1 = 0
for _ in range(realizations):
    x = contagion_process(A, gamma, c1, x0, tmin=0, tmax=tmax)
    ipn_c1 += infections_per_node(x, mode) / realizations

f = lambda b: ipn_func(b, ipn_c1, cf2, gamma, A, rho0, 100, tmax, mode)
b2, bvec2, fvec2 = robbins_monro_solve(
    f, b0, a, alpha, max_iter, tol, verbose=True, loss="function", return_values=True
)

# max fitting
mode = "max"
ipn_c1 = 0
for _ in range(realizations):
    x = contagion_process(A, gamma, c1, x0, tmin=0, tmax=tmax)
    ipn_c1 += infections_per_node(x, mode) / realizations

f = lambda b: ipn_func(b, ipn_c1, cf2, gamma, A, rho0, 100, tmax, mode)
b3, bvec3, fvec3 = robbins_monro_solve(
    f, b0, a, alpha, max_iter, tol, verbose=True, loss="function", return_values=True
)

In [ ]:
plt.figure()
plt.subplot(121)
plt.plot(bvec1, label="mean")
plt.plot(bvec2, label="median")
plt.plot(bvec3, label="max")
plt.xlabel("iterations")
plt.ylabel("fitted probability")

plt.subplot(122)
plt.plot(fvec1, label="mean")
plt.plot(fvec2, label="median")
plt.plot(fvec3, label="max")
plt.legend()
plt.xlabel("iterations")
plt.ylabel("diff. between expected IPNs")
plt.tight_layout()

plt.savefig("test.png", dpi=1000)